In [ ]:
# In [agents.py]:

# agents.py

import json
import os
import logging
from typing import Dict, Any, Optional

logger = logging.getLogger(__name__)

class AgentManager:
    def __init__(self, file_path: str = "agents.json"):
        self.file_path = file_path
        self.agents = self.load_agents()

    def load_agents(self) -> Dict[str, Dict[str, Any]]:
        try:
            if os.path.exists(self.file_path):
                with open(self.file_path, "r") as f:
                    return json.load(f)
            return {}
        except json.JSONDecodeError as e:
            logger.error(f"Error loading agents from {self.file_path}: {e}")
            return {}

    def save_agents(self) -> None:
        try:
            with open(self.file_path, "w") as f:
                json.dump(self.agents, f, indent=2)
        except IOError as e:
            logger.error(f"Error saving agents to {self.file_path}: {e}")

    def create_agent(self, name: str, instruction: str, model_config: Dict[str, Any]) -> bool:
        if name in self.agents:
            logger.warning(f"Agent '{name}' already exists")
            return False
        self.agents[name] = {"instruction": instruction, "model_config": model_config}
        self.save_agents()
        return True

    def get_agent(self, name: str) -> Optional[Dict[str, Any]]:
        return self.agents.get(name)

    def list_agents(self) -> list:
        return list(self.agents.keys())

    def update_agent(self, name: str, instruction: Optional[str] = None, model_config: Optional[Dict[str, Any]] = None) -> bool:
        if name not in self.agents:
            logger.warning(f"Agent '{name}' does not exist")
            return False
        if instruction is not None:
            self.agents[name]["instruction"] = instruction
        if model_config is not None:
            self.agents[name]["model_config"] = model_config
        self.save_agents()
        return True

    def delete_agent(self, name: str) -> bool:
        if name not in self.agents:
            logger.warning(f"Agent '{name}' does not exist")
            return False
        del self.agents[name]
        self.save_agents()
        return True


In [ ]:
# In [combine_py_files_to_new_notebook.py]:

import os
import nbformat as nbf
from nbformat.v4 import new_notebook, new_code_cell

def combine_py_files_to_notebook(directory_path, output_file):
    # Create a new notebook
    nb = new_notebook()
    
    # Iterate through all .py files in the directory
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith('.py'):
            file_path = os.path.join(directory_path, filename)
            
            # Read the content of the .py file
            with open(file_path, 'r') as file:
                content = file.read()
            
            # Create a new code cell with the file content
            cell_content = f"# {filename}\n\n{content}"
            cell = new_code_cell(cell_content)
            
            # Add the cell to the notebook
            nb.cells.append(cell)
    
    # Write the notebook to a file
    with open(output_file, 'w') as f:
        nbf.write(nb, f)
    
    def process_files(input_files, output_file):
        nb = new_notebook()
        
        for file_path in input_files:
            if os.path.isfile(file_path) and file_path.endswith('.py'):
                with open(file_path, 'r') as file:
                    content = file.read()
                
                filename = os.path.basename(file_path)
                cell_content = f"# {filename}\n\n{content}"
                cell = new_code_cell(cell_content)
                nb.cells.append(cell)
        
        with open(output_file, 'w') as f:
            nbf.write(nb, f)
    
    if __name__ == "__main__":
        import sys
        
        if len(sys.argv) < 2:
            print("Usage: python combine_py_files_to_new_notebook.py <input_file1> [input_file2 ...] <output_file>")
            sys.exit(1)
        
        output_file = sys.argv[-1]
        input_files = sys.argv[1:-1]
        
        if len(input_files) == 1 and os.path.isdir(input_files[0]):
            combine_py_files_to_notebook(input_files[0], output_file)
        else:
            process_files(input_files, output_file)
        
        print(f"Notebook '{output_file}' has been created successfully.")
    
    # Iterate through all .py files in the directory
    for filename in sorted(os.listdir(directory_path)):
        if filename.endswith('.py'):
            file_path = os.path.join(directory_path, filename)
            
            # Read the content of the .py file
            with open(file_path, 'r') as file:
                content = file.read()
            
            # Create a new code cell with the file content
            cell_content = f"# {filename}\n\n{content}"
            cell = new_code_cell(cell_content)
            
            # Add the cell to the notebook
            nb.cells.append(cell)
    
    # Write the notebook to a file
    with open(output_file, 'w') as f:
        nbf.write(nb, f)

# Usage
directory_path = '.'  # Current directory, change this if needed
output_file = 'combined_scripts.ipynb'

combine_py_files_to_notebook(directory_path, output_file)
print(f"Notebook '{output_file}' has been created successfully.")

In [ ]:
# In [config.py]:

# Begin Groups of Modifiers, grouped by type or style of task and content being worked on.
# On run, modifiers are appended to the end of the latest ai response and fed back to the AI to either critique or refine its previous response.

# This modifier group is generalized ideation intialization
from collections import deque
from dotenv import load_dotenv
import os
# Constants
MAX_CHAT_HISTORY_LENGTH = 14
MAX_RETRIES = 3
BACKOFF_FACTOR = 2
# Constants

SAFETY_SETTINGS = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
]


load_dotenv()
#LLM
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')

#SEARCH APIS
GOOGLE_CUSTOM_SEARCH_API_KEY = os.getenv('GOOGLE_CUSTOM_SEARCH_API_KEY')
GOOGLE_CUSTOM_SEARCH_ENGINE_ID = os.getenv('GOOGLE_CUSTOM_SEARCH_ENGINE_ID')
BRAVE_SEARCH_API_KEY = os.getenv('BRAVE_SEARCH_API_KEY')
HF_TOKEN = os.getenv('HF_TOKEN')


# Constants
MAX_TRUNCATE_LENGTH = 5000 #Length of truncated chat_log items
MAX_INVALID_ATTEMPTS = 3
MAX_SEARCH_RESULTS = 8
MAX_CONTENT_LENGTH = 4000  # Maximum number of characters to extract from each webpage
TIMEOUT = 15
MAX_SEARCH_QUERIES_PER_REQUEST = 2



In [ ]:
# In [extract_text_from_html.py]:

import requests
from bs4 import BeautifulSoup
import re
import gzip
import io
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager

def extract_text_from_html(url, extract_outside_main=False, extract_comments=False):
    # Request with headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
        'DNT': '1',
    }
    response = requests.get(url, headers=headers)

    content_type = response.headers.get('Content-Type', '')
    if 'text/html' not in content_type:
        raise ValueError(f"Expected HTML, but received {content_type}")

    # Handle gzip or plain text
    if response.headers.get('Content-Encoding') == 'gzip':
        try:
            html_content = gzip.decompress(response.content).decode('utf-8')
        except (gzip.BadGzipFile, OSError):
            html_content = response.text
    else:
        html_content = response.text

    soup = BeautifulSoup(html_content, 'html.parser')
    main_content = soup.find(['div', 'main', 'article'], class_=re.compile(r'(main|content)', re.I)) or soup.body
    main_text = main_content.get_text(separator=' ', strip=True) if main_content else ''

    # Fall back to Selenium if main_text is invalid
    if len(main_text) < 200 and ('cookies' in main_text.lower() or 'javascript' in main_text.lower() or len(main_text) < 10):
        main_text = extract_with_selenium(url)

    # Optionally extract outside text and comments
    outside_text = extract_outside_main_content(soup, main_content) if extract_outside_main else ''
    comments = extract_comments_section(soup) if extract_comments else ''

    # Combine extracted content
    return clean_text(f"{main_text} {outside_text} {comments}".strip())

def extract_outside_main_content(soup, main_content):
    outside_tags = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4'])
    return ' '.join(tag.get_text(strip=True) for tag in outside_tags if tag not in main_content.descendants)

def extract_comments_section(soup):
    comment_section = soup.find('div', class_='comments')
    return comment_section.get_text(separator=' ', strip=True) if comment_section else ''

def extract_with_selenium(url):
    # Set up Edge options for headless browsing with human-like behavior
    edge_options = Options()
    edge_options.add_argument("--headless")
    edge_options.add_argument("--disable-gpu")
    edge_options.add_argument("--no-sandbox")

    # Human-like headers and user-agent simulation
    edge_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    edge_options.add_argument("accept=text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8")
    edge_options.add_argument("accept-language=en-US,en;q=0.9")
    edge_options.add_argument("accept-encoding=gzip, deflate, br")

    # Set up the WebDriver for Edge
    service = Service(EdgeChromiumDriverManager().install())
    driver = webdriver.Edge(service=service, options=edge_options)

    # Load the page with Selenium
    driver.get(url)

    # Wait for JavaScript to execute
    driver.implicitly_wait(10)

    # Get the page source after JavaScript execution
    html_content = driver.page_source

    # Close the browser
    driver.quit()

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')

    # Extract main content
    main_content = soup.find(['div', 'main', 'article'], class_=re.compile(r'(main|content)', re.I)) or soup.body
    return main_content.get_text(separator=' ', strip=True) if main_content else ''


def clean_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)

    # Remove duplicate/multiple adjacent new lines and white space characters
    text = re.sub(r'\s+', ' ', text)

    # Remove any symbols other than punctuation and dollar signs
    text = re.sub(r'[^\w\s.,$]', '', text)

    return text.strip()


# Example usage
url = 'https://www.msn.com/en-us/news/world/middle-east-conflict-live-updates-israel-strikes-lebanon-after-hezbollah-vows-to-retaliate/ar-AA1qSCMd?ocid=msedgntp&pc=U531&cvid=ddf35d5b73ce485be5947960b58139d7&ei=44'
extracted_text = extract_text_from_html(url, extract_outside_main=False, extract_comments=False)
print(extracted_text)

with open('extracted_text.txt', 'w', encoding='utf-8') as f:
    f.write(extracted_text)


In [ ]:
# In [gui.py]:

# gui.py

import tkinter as tk
from tkinter import ttk, scrolledtext, Menu, filedialog, simpledialog, messagebox
import logging
from typing import List, Optional, Tuple, Dict
from functools import partial

from models import ModelManager, ModelFactory, generate_convo_context
from agents import AgentManager
from search_manager import SearchManager, SearchAPI, DuckDuckGoSearchProvider
from config import MAX_SEARCH_RESULTS
import json
logger = logging.getLogger(__name__)

class ModifierTreeView(ttk.Treeview):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.selected_modifiers = []
        self.bind("<<TreeviewSelect>>", self.on_select)

    def on_select(self, event):
        self.selected_modifiers = []
        for item in self.selection():
            item_text = self.item(item, "text")
            if item_values := self.item(item, "values"):
                self.selected_modifiers.append((item_text, item_values[0]))

class App(tk.Tk):
    modifier_groups: Dict[str, Dict[str, str]] = {
        "SONG WRITING": {
            "Brainstorm Concepts": "Based on the above context and using it as inspiration, generate a list of potential concepts that could be developed into compelling lyrics for a song",
            "Narrow Down and Select Concept": "Evaluate each concept from the above list and determine which one(s) are the best quality by playing around with some potential verses, hooks, wordplay, you could use for each one and optionally mixing and matching concepts to form complex combinations of concepts, and at the end of your experimentation, decide the best one you choose for developing into a full fledged song"
        },
        "JOKE WRITING": {
            "Comedy Technique": "Analyze the current content and suggest three different comedy techniques that could be applied to enhance its humor. Provide brief examples for each technique.",
            "Punch-up": "Review the existing content and punch-up the dialogue or descriptions to make them snappier, funnier, and more engaging. Focus on adding unexpected twists, wordplay, or exaggeration where appropriate."
        }
    }

    def __init__(self, search_manager: SearchManager, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.title("Creative AI Assistant")
        self.geometry("800x600")

        self.model_manager = ModelManager(search_enabled=True)
        self.agent_manager = AgentManager()
        self.search_manager = search_manager 

        self.chat_log: List[str] = []
        self.context = ""
        self.current_prompt = ""
        self.last_output = ""

        self.setup_ui()
        self.setup_menu()

    def setup_ui(self):
        self.main_frame = ttk.Frame(self)
        self.main_frame.pack(side="left", fill="both", expand=True)

        self.sidebar_frame = ttk.Frame(self)
        self.sidebar_frame.pack(side="right", fill="y")

        self.chat_history = scrolledtext.ScrolledText(self.main_frame, wrap="word")
        self.chat_history.pack(fill="both", expand=True)

        self.user_prompt = ttk.Entry(self.main_frame)
        self.user_prompt.pack(fill="x")
        self.user_prompt.bind("<Return>", self.run_workflow)

        self.setup_sidebar()

    def setup_sidebar(self):
        self.model_label = ttk.Label(self.sidebar_frame, text="Model:")
        self.model_label.pack()

        self.model_var = tk.StringVar(value="assistant")
        self.model_options = ["brainstorm", "assistant", "director", "prompter", "researcher", "critic"]
        self.model_dropdown = ttk.OptionMenu(self.sidebar_frame, self.model_var, *self.model_options)
        self.model_dropdown.pack()

        self.workflow_label = ttk.Label(self.sidebar_frame, text="Workflow:")
        self.workflow_label.pack()
        self.workflow_var = tk.StringVar(value="Optimator")
        self.workflow_options = ["Optimator"]
        self.workflow_dropdown = ttk.OptionMenu(self.sidebar_frame, self.workflow_var, *self.workflow_options)
        self.workflow_dropdown.pack()

        self.iterations_label = ttk.Label(self.sidebar_frame, text="Iterations:")
        self.iterations_label.pack()
        self.iterations_var = tk.IntVar(value=1)
        self.iterations_options = list(range(1, 21))
        self.iterations_dropdown = ttk.OptionMenu(self.sidebar_frame, self.iterations_var, 1, *self.iterations_options)
        self.iterations_dropdown.pack()

        # Create modifier tree view
        self.modifier_tree = ModifierTreeView(self.sidebar_frame, selectmode="extended")
        self.modifier_tree.pack(fill="both", expand=True)
        self.populate_modifier_tree()

        # Add "Run Modifier Groups" button
        self.run_modifiers_button = ttk.Button(self.sidebar_frame, text="Run Modifier Groups", command=self.run_modifier_groups)
        self.run_modifiers_button.pack()

        self.search_enabled = tk.BooleanVar(value=True)
        self.search_checkbox = ttk.Checkbutton(self.sidebar_frame, text="Enable Search", variable=self.search_enabled)
        self.search_checkbox.pack()

        self.improve_prompt = tk.BooleanVar(value=False)
        self.improve_prompt_checkbox = ttk.Checkbutton(self.sidebar_frame, text="Automatically Improve Prompt", variable=self.improve_prompt)
        self.improve_prompt_checkbox.pack()

        self.brainstorm_enabled = tk.BooleanVar(value=False)
        self.brainstorm_checkbox = ttk.Checkbutton(self.sidebar_frame, text="Enable Brainstorm", variable=self.brainstorm_enabled)
        self.brainstorm_checkbox.pack()

        self.run_button = ttk.Button(self.sidebar_frame, text="Run Workflow", command=self.run_workflow)
        self.run_button.pack()

        self.single_model_button = ttk.Button(self.sidebar_frame, text="Run Single Model", command=self.run_single_model)
        self.single_model_button.pack()

        self.manage_agents_button = ttk.Button(self.sidebar_frame, text="Manage Agents", command=self.manage_agents)
        self.manage_agents_button.pack()

        self.agent_var = tk.StringVar(value="")
        self.agent_dropdown = ttk.OptionMenu(self.sidebar_frame, self.agent_var, "", *self.agent_manager.list_agents())
        self.agent_dropdown.pack()

    def setup_menu(self):
        menubar = Menu(self)
        filemenu = Menu(menubar, tearoff=0)
        filemenu.add_command(label="Save Chat Log", command=self.save_chat_log)
        filemenu.add_separator()
        filemenu.add_command(label="Exit", command=self.quit)
        menubar.add_cascade(label="File", menu=filemenu)
        self.config(menu=menubar)

    def populate_modifier_tree(self):
        for group, modifiers in self.modifier_groups.items():
            group_id = self.modifier_tree.insert("", "end", text=group)
            for modifier_title, modifier_text in modifiers.items():
                self.modifier_tree.insert(group_id, "end", text=modifier_title, values=(modifier_text,))


    def save_chat_log(self):
        if file_path := filedialog.asksaveasfilename(
            defaultextension=".txt", filetypes=[("Text Files", "*.txt")]
        ):
            try:
                with open(file_path, "w") as file:
                    file.write(self.chat_history.get("1.0", "end"))
                logger.info(f"Chat log saved to {file_path}")
            except IOError as e:
                logger.error(f"Error saving chat log: {e}")
                messagebox.showerror("Error", f"Failed to save chat log: {e}")

    def run_modifier_groups(self):
        selected_modifiers = self.modifier_tree.selected_modifiers
        if not selected_modifiers:
            messagebox.showwarning("No Modifiers Selected", "Please select at least one modifier before running.")
            return

        user_prompt = self.user_prompt.get()
        if not user_prompt and self.last_output:
            user_prompt = self.last_output

        if not user_prompt:
            messagebox.showwarning("No Input", "Please enter some text or generate content before running modifiers.")
            return

        self.update_chat_history("User:", user_prompt)
        self.process_modifier_groups(user_prompt, selected_modifiers)

    def process_modifier_groups(self, user_prompt: str, modifiers: List[Tuple[str, str]]):
        for modifier_title, modifier_text in modifiers:
            self.update_chat_history("Modifier:", f"{modifier_title}: {modifier_text}", "thinking")
            model_output = self.model_manager.generate_response(
                "assistant",
                f"{user_prompt}\n\nModifier: {modifier_text}",
                self.chat_log,
                self.context,
                self.search_manager
            )
            self.update_chat_history("Assistant:", model_output)
            user_prompt = model_output

        self.last_output = user_prompt
        self.modifier_tree.selection_remove(self.modifier_tree.selection())

    def manage_agents(self):
        action = simpledialog.askstring(
            "Manage Agents",
            "Enter 'create' to create a new agent or 'list' to see existing agents:"
        )
        if action == "create":
            self.create_agent()
        elif action == "list":
            self.list_agents()

    def create_agent(self):
        name = simpledialog.askstring("Create Agent", "Enter agent name:")
        if not name:
            return
        instruction = simpledialog.askstring("Create Agent", "Enter agent instruction:")
        if not instruction:
            return
        model_config = simpledialog.askstring("Create Agent", "Enter model configuration (JSON format):")
        try:
            model_config = json.loads(model_config)
        except json.JSONDecodeError:
            messagebox.showerror("Error", "Invalid JSON for model configuration")
            return

        if self.agent_manager.create_agent(name, instruction, model_config):
            messagebox.showinfo("Success", f"Agent '{name}' created successfully")
            self.update_agent_dropdown()
        else:
            messagebox.showerror("Error", f"Agent '{name}' already exists")

    def list_agents(self):
        if agents := self.agent_manager.list_agents():
            messagebox.showinfo("Agents", "\n".join(agents))
        else:
            messagebox.showinfo("Agents", "No agents created yet")

    def update_agent_dropdown(self):
        menu = self.agent_dropdown["menu"]
        menu.delete(0, "end")
        for agent in self.agent_manager.list_agents():
            menu.add_command(label=agent, command=partial(self.agent_var.set, agent))

    def run_single_model(self):
        user_prompt = self.user_prompt.get()
        if not user_prompt:
            return

        selected_model = self.model_var.get()
        selected_agent = self.agent_var.get()

        self.update_chat_history("User:", user_prompt)
        self.context = generate_convo_context(user_prompt, self.chat_log)
        self.process_single_model(selected_model, user_prompt, selected_agent)

    def process_single_model(self, model_type: str, user_prompt: str, agent_name: Optional[str] = None):
        self.update_chat_history(f"{model_type.capitalize()}:", "Processing...", "thinking")

        if agent_name:
            if agent := self.agent_manager.get_agent(agent_name):
                model = ModelFactory.create_model(agent["instruction"], **agent["model_config"])
                model_output = model.generate_content(user_input=user_prompt)
                model_output = model_output.text if model_output.text else ""
            else:
                model_output = "Error generating model output"
        else:
            model_output = self.model_manager.generate_response(
                model_type, user_prompt, self.chat_log, self.context, self.search_manager
            )

        self.update_chat_history(f"{model_type.capitalize()}:", model_output)
        self.last_output = model_output

    def run_workflow(self, event=None):
        user_prompt = self.user_prompt.get()
        self.user_prompt.delete(0, tk.END)

        if not user_prompt and self.last_output:
            user_prompt = self.last_output

        if not user_prompt:
            return

        self.update_chat_history("User:", user_prompt)
        self.current_prompt = user_prompt
        self.context = generate_convo_context(self.current_prompt, self.chat_log)

        workflow = self.workflow_var.get()
        self.process_workflow(workflow, user_prompt)

    def process_workflow(self, workflow: str, user_prompt: str):
        if workflow == "Optimator":
            self.run_optimator_workflow(user_prompt)
        if workflow == "Code Generator":
            self.run_code_generator_workflow(user_prompt)
            
    def run_code_generator_workflow(self, user_prompt: str):
        if self.improve_prompt.get():
            self.update_chat_history("Prompter:", "Improving prompt...", "thinking")
        # Add more workflows here as needed

    def run_optimator_workflow(self, user_prompt: str):
        if self.improve_prompt.get():
            self.update_chat_history("Prompter:", "Improving prompt...", "thinking")
            self.generate_convo_context(user_prompt, self.chat_log[-10 or len(self.chat_log):])
            improved_prompt = self.model_manager.generate_response(
                "prompter", user_prompt, self.chat_log, self.context, self.search_manager
            )
            user_prompt = improved_prompt
            self.update_chat_history("Prompter:", user_prompt)
        if self.brainstorm_enabled.get():
            self.update_chat_history("Brainstorm:", "Generating ideas...", "thinking")
            brainstorm_output = self.model_manager.generate_response(
                "brainstorm", user_prompt, self.chat_log, self.context, self.search_manager
            )
            user_prompt += f"\n\nBrainstorm ideas:\n{brainstorm_output}"
            self.update_chat_history("Brainstorm:", brainstorm_output)

        iterations = 0
        total_iterations = self.iterations_var.get()
        while iterations < total_iterations:
            self.update_chat_history("writer:", "Processing...", "thinking")
            writer_output = self.model_manager.generate_response(
                "writer", user_prompt, self.chat_log, self.context, self.search_manager
            )
            self.update_chat_history("writer:", writer_output)
            self.last_output = writer_output

            self.update_chat_history("Critic:", "Evaluating...", "thinking")
            critic_output = self.model_manager.generate_response(
                "critic", writer_output, self.chat_log, self.context, self.search_manager
            )
            self.update_chat_history("Critic:", critic_output)

            self.update_chat_history("Director:", "Planning next steps...", "thinking")
            director_output = self.model_manager.generate_response(
                "director", critic_output, self.chat_log, self.context, self.search_manager
            )
            self.update_chat_history("Director:", director_output)
            iterations += 1

    def update_chat_history(self, speaker: str, message: str, message_type: str = "normal"):
        """
        Updates the chat history with a new message from the specified speaker.

        Args:
            speaker (str): The name of the person sending the message.
            message (str): The content of the message.
            message_type (str, optional): Type of the message. Defaults to "normal".
                If set to "thinking", it will be displayed without a newline character at the end.
        """
        # Make the chat history widget editable
        self.chat_history.config(state="normal")

        # Insert the new message into the chat history based on its type
        if message_type == "thinking":
            # For 'thinking' messages, insert with no trailing newline character
            self.chat_history.insert(tk.END, f"{speaker} {message}\n")
        else:
            # For other types of messages, insert with a newline character at the end
            self.chat_history.insert(tk.END, f"{speaker} {message}\n\n")

        # Make the chat history widget read-only again
        self.chat_history.config(state="disabled")

        # Scroll the chat history to the bottom to show the new message
        self.chat_history.see(tk.END)

        # Trigger an update to refresh the UI (e.g., in case of a 'thinking' message)
        self.update()

    def run(self):
        self.mainloop()


In [ ]:
# In [main.py]:

# main.py
import logging
import tkinter as tk
from tkinter import ttk, scrolledtext, Menu, filedialog, simpledialog, messagebox

from config import GEMINI_API_KEY, MAX_SEARCH_RESULTS
from models import ModelManager
from agents import AgentManager
from search_manager import SearchManager, SearchAPI, DuckDuckGoSearchProvider, SearchProviderFactory
from gui import App

# Set up logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

class AIAssistantApp:
    def __init__(self):
        self.model_manager = ModelManager(search_enabled=True)
        self.agent_manager = AgentManager()
        google_api = SearchProviderFactory.create_provider("google")
        duckduckgo_provider = SearchProviderFactory.create_provider("duckduckgo")
        self.search_manager = SearchManager([google_api], duckduckgo_provider)
        self.app = App(self.search_manager)
        self.app.model_manager = self.model_manager
        self.app.agent_manager = self.agent_manager

    def initialize_search_manager(self):
        # Add more APIs like Brave using the factory if needed
        return self.search_manager  # Return the search_manager instance

    def run(self):
        try:
            self.app.mainloop()
        except Exception as e:
            logger.error(f"Unexpected error in main loop: {e}")
        finally:
            logger.info("Application closed")
            
def main():
    try:
        app = AIAssistantApp()
        app.run()
    except Exception as e:
        logger.error(f"Fatal error: {e}")
        messagebox.showerror("Fatal Error", f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()


In [ ]:
# In [models.py]:

# models.py

import logging
from functools import lru_cache
from typing import Dict, Any, List, Optional
import google.generativeai as genai

from config import GEMINI_API_KEY, SAFETY_SETTINGS, MAX_SEARCH_RESULTS
from search_manager import SearchManager

logger = logging.getLogger(__name__)

class ModelFactory:
    @staticmethod
    @lru_cache(maxsize=None)
    def initialize_model(
        system_instruction: str,
        model_name: str = "models/gemini-1.5-flash-latest",
        **generation_config: Any
    ) -> genai.GenerativeModel:
        try:
            genai.configure(api_key=GEMINI_API_KEY)
            return genai.GenerativeModel(
                system_instruction=system_instruction,
                model_name=model_name,
                generation_config=generation_config,
                safety_settings=SAFETY_SETTINGS,
            )
        except Exception as e:
            logger.error(f"Error initializing model: {e}")
            raise

    @classmethod
    def create_model(cls, instruction: str, **config: Any) -> genai.GenerativeModel:
        return cls.initialize_model(instruction, **config)

class ModelManager:
    MODEL_CONFIGS: Dict[str, tuple] = {
        "brainstorm": (
            "You are a spontaneous, ideation machine with no limits that always seems to connect disparate ideas and concepts into diverse and novel creative solutions and content.",
            {"temperature": 1.0, "top_p": 0.9, "top_k": 60, "max_output_tokens": 32000},
        ),
        "writer": (
            """
            *** WRITER'S HANDBOOK: MASTER SYSTEM INSTRUCTIONS ***
            These are the guidelines and instructions that guide your writing.
                        
            I. ROLE & CONTEXT:
            You are a dynamic, innovative, and linguistically skilled content creator that analyzes and fully understands the context of the conversation to determine if creativity or factual information is required of you, adapting to perform various roles and specialties as provided/required. Your primary objective is to produce exceptional writing outputs that precisely aligns with the user's requirements and the specified goal.  
            
            II. RESPONSIBILITIES AND REQUIREMENTS:
            1. Analyze the user's request and adapt your writing style, tone, and expertise accordingly.
            2. Synthesize complex information from various sources to create insightful, value-added content.
            3. Craft engaging narratives, dialogues, and descriptions that captivate the audience.
            4. Verify that all claims and statistics are accurate and properly sourced.
            5. Depending on the current work, employ creativity and maintain factual accuracy, tailoring your approach to the content type.
            5. Continuously refine and optimize your output based on all feedback and any evolving requirements.
            6. Adaptively switch between different roles, writing styles, tones, and subject matter expertise based on the context.
            7. Adhere to guidelines, directions and any brand voices as provided by the user and/or director.
            8. Ensure the final output meets or exceeds the user's expectations and project requirements.
            
            III. GOALS AND GUIDELINES:
            Unless directed otherwise, your ultimate goal is to generate and develop content to be optimally:
            1. Comprehensive: Deliver thorough, well-researched content that covers all relevant aspects of the topic.
            2. Cogent: Present ideas logically and coherently, avoiding ambiguity and ensure your content is accessible to the target audience while maintaining depth, sophistication, and persuasiveness.
            3. Concise: Eliminate redundancy and focus on essential information.
            4. Creative: Generate original ideas and unique perspectives while maintaining accuracy.
            Content Enhancement:
            5. Contributional: Identify opportunities to expand upon or challenge conventional ideas within the subject matter and contribute novel insights, introduce a fresh perspective, and/or build upon existing information/understanding to further the global discourse or advance a field of study.
    
            By fulfilling these responsibilities, you will consistently deliver high-quality, tailored content that meets the diverse needs of users across various domains and writing tasks.""",    
            {"temperature": 0.8, "top_p": 0.9, "top_k": 50, "max_output_tokens": 32000},
        ),
        "director": (
            """You are the director in a creative think tank with a content producer, a critic, a researcher, and a brainstormer. As the director, you are responsible for coordinating the workflow of any given project/task the think tank team is assigned. Your specific responsibilities are to: keep track of progress, keep the team on task, evaluate the team's latest efforts and the critic's feedback/suggestions in brief status reports, and direct the workflow by providing guidance on what to work on next after you provide your status reports. Be efficient and prioritize directness and clarity over politeness when giving feedback.""",
            {"temperature": 0.3, "top_p": 0.8, "top_k": 30, "max_output_tokens": 32000},
        ),
        "prompter": (
            """The assistant is Prompt Optimizer. It will develop and enhance the user input prompt by: 1. Splitting complex tasks into smaller actionable subtasks and outlining the step by step procedure to follow to achieve the best outcome, adding relevant details and context in alignment with the user's intent and desired output. 2. Maximizing clarity and minimizing ambiguity to prevent misinterpretations and uncertainty. 3. Providing coherent, cohesive and comprehensive detail. Begin your response with "Objective(s) and Instruction(s):" followed by the enhanced and optimized prompt only and nothing else (do not include any other conversational verbage or chatter).""",
            {"temperature": 0.4, "top_p": 0.8, "top_k": 30, "max_output_tokens": 32000},
        ),
        "critic": (
            "The assistant is referred to as 'critic'. The critic's mission is to provide constructive feedback on the latest version of the work in progress. The critic's feedback should be clear, actionable, and aimed at enhancing the quality of the final output. The critic should highlight strengths, weaknesses, and areas for improvement. When possible, the critic should provide specifics and examples. If no objective improvements can be made, the critic should state so without providing feedback.",
            {"temperature": 0.6, "top_p": 0.8, "top_k": 40, "max_output_tokens": 32000},
        ),
        "researcher": (
            "The assistant is referred to as the 'researcher'. The researcher's primary goal is to extract relevant information from search results and present it in a report for the requesting team member to provide missing/requested information based upon the context of the conversation / message log. !!! IT DOES NOT ASSUME OR MAKE UP FACTS. IT DOES NOT PROVIDE ANY INFORMATION TO THE TEAM THAT IS NOT PRESENT IN THE SEARCH RESULT CONTENT !!!",
            {"temperature": 0.3, "top_p": 0.7, "top_k": 30, "max_output_tokens": 32000},
        ),
    }

    def __init__(self, search_enabled: bool = True):
        self.search_enabled = search_enabled

    def get_model(self, model_type: str) -> genai.GenerativeModel:
        if model_type not in self.MODEL_CONFIGS:
            raise ValueError(f"Unknown model type: {model_type}")
        instruction, config = self.MODEL_CONFIGS[model_type]

        if model_type in {"writer", "researcher"} and self.search_enabled:
            instruction += self.get_search_instructions()

        model_name = "models/gemini-1.5-pro-latest" if model_type == "writer" else "models/gemini-1.5-flash-latest"
        return ModelFactory.create_model(instruction, model_name=model_name, **config)

    @staticmethod
    def get_search_instructions() -> str:
        return """
        It is important that you double check your work against the latest available information online when making any claim and assertion of which you are not 110-percent certain. When necessary to form a proper response/output, request web searches to verify information or gather additional details and use one or up to two specific, detailed search queries in the following format explicitly:
        ```
        SEARCH_QUERIES: | Query 1 | Query 2 |
        ```
        *IF requesting a web search, you MUST use the above format exactly otherwise it will go unseen.
        You will receive the text or relevant information from top search results' pages in a subsequent message and then you can integrate the relevant information into and complete your original/actual full response.
        """

    def generate_response(
        self,
        model_type: str,
        user_prompt: str,
        chat_log: List[str],
        context: str,
        search_manager: SearchManager
    ) -> str:
        try:
            model = self.get_model(model_type)
            generate_convo_context(user_prompt, chat_log[-10:])
            full_prompt = f"{context}\n{user_prompt}"

            response = model.generate_content(full_prompt)
            response_text = response.text if response.text else ""

            if self.search_enabled and "SEARCH_QUERIES:" in response_text:
                main_response, search_queries = self.parse_web_search_request(response_text)
                chat_log.append(f"{model_type.capitalize()}: {response_text}")
                search_output = self.begin_web_search(chat_log, context, search_queries, search_manager)

                updated_prompt = f"{full_prompt}\n\nAdditional Information from Search Results:\n{search_output}"
                updated_response = model.generate_content(updated_prompt)
                response_text = updated_response.text if updated_response.text else ""

            chat_log.append(f"{model_type.capitalize()}: {response_text}")
            return response_text
        except Exception as e:
            logger.error(f"Error generating response for {model_type}: {e}")
            return f"An error occurred while generating the response: {str(e)}"

    @staticmethod
    def parse_web_search_request(response_text: str) -> tuple:
        parts = response_text.split("SEARCH_QUERIES:")
        if len(parts) > 1:
            search_queries = [query.strip() for query in parts[1].split("|") if query.strip()]
            main_response = parts[0].strip()
            if len(search_queries) > 2:
                search_queries = search_queries[:2]
            return main_response, search_queries
        return response_text, []

    def begin_web_search(
        self,
        chat_log: List[str],
        context: str,
        search_queries: List[str],
        search_manager: SearchManager,
        prompt: str = ""
    ) -> str:
        context = self.generate_convo_context(chat_log, prompt)
        results = []
        for query in search_queries:
            search_results = search_manager.search(query, num_results=MAX_SEARCH_RESULTS)
            results.extend(
                f"**{result['title']}** ({result['url']})\n{result['content']}\n"
                for result in search_results
            )

        researcher_model = self.get_model("researcher")
        research_prompt = f"{context}\n\nBased on the context/conversation history and search query above, analyze the following search results and from them synthesize a relevant, useful, and comprehensive while succinct report that addresses and answers the searched query:\n\n{''.join(results)}"
        research_response = researcher_model.generate_content(research_prompt)
        return research_response.text if research_response.text else "No research findings."

def generate_convo_context(prompt: str, chat_log: List[str]) -> str:
        return f"\nLatest Progress: {'\n'.join(chat_log[-10:])}\nTarget final output and/or instruction from the user: {prompt}"

In [ ]:
# In [pyfi2nb.py]:

import os
import nbformat as nbf
from nbformat.v4 import new_notebook, new_code_cell
import argparse

def create_code_cell(file_path):
    """
    Create a new code cell with content from the file.

    Args:
        file_path (str): Path to the file.

    Returns:
        nbformat.v4.new_code_cell: A new code cell with content and filename.
    """
    try:
        with open(file_path, 'r') as file:
            content = file.read()
    except FileNotFoundError as e:
        raise ValueError(f"File '{file_path}' not found.") from e
    except PermissionError as e:
        raise PermissionError(f"Permission denied for file '{file_path}'.") from e

    filename = os.path.basename(file_path)
    cell_content = f"# In [{filename}]:\n\n{content}"
    return new_code_cell(cell_content)

def combine_files_to_notebook(input_files=None, output_file="project_notebook_full.ipynb"):
    print(f"Current working directory: {os.getcwd()}")
    if input_files is None:
        input_files = [f for f in os.listdir() if f.endswith('.py')]
        print(f"Python files found in current directory: {input_files}")
    """
    Combine Python files into a Jupyter notebook.

    Args:
        input_files (list, optional): List of file paths or a single directory path. If None, all .py files in the current directory are used.
        output_file (str, optional): Output notebook file path. Defaults to 'project_notebook_full.ipynb'.
    """
    if input_files is None:
        input_files = [f for f in os.listdir() if f.endswith('.py')]
        print(f"Using all .py files in the current directory: {input_files}")

    nb = new_notebook()
    valid_inputs = []

    for file_path in input_files:
        if os.path.isdir(file_path):
            py_files = [f for f in sorted(os.listdir(file_path)) if f.endswith('.py')]
            for py_file in py_files:
                full_path = os.path.join(file_path, py_file)
                try:
                    cell = create_code_cell(full_path)
                    nb.cells.append(cell)
                    valid_inputs.append(full_path)
                except ValueError as ve:
                    print(f"Error processing {full_path}: {ve}")
                except PermissionError as pe:
                    print(f"Permission error for {full_path}: {pe}")
        elif os.path.isfile(file_path) and file_path.endswith('.py'):
            try:
                cell = create_code_cell(file_path)
                nb.cells.append(cell)
                valid_inputs.append(file_path)
            except ValueError as ve:
                print(f"Error processing {file_path}: {ve}")
            except PermissionError as pe:
                print(f"Permission error for {file_path}: {pe}")
        else:
            print(f"Skipping invalid file/directory: {file_path}")

    if not valid_inputs:
        print("No valid input files found. Exiting.")
        return

    nbf.write(nb, output_file)
    print(f"Notebook '{output_file}' created successfully using: {', '.join(valid_inputs)}")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Combine Python scripts into a Jupyter notebook.')
    parser.add_argument('input', nargs='*', help='Input file paths or a directory containing Python scripts. Defaults to all .py files in the current directory.')
    parser.add_argument('-o', '--output', default="project_notebook_full.ipynb", help='Output notebook file path.')
    args = parser.parse_args()

    print(f"Input arguments: {args.input}")
    
    # Combine files and create the notebook
    combine_files_to_notebook(args.input if args.input else None, args.output)
		

In [ ]:
# In [search_manager.py]:

import requests
from bs4 import BeautifulSoup, Comment
import time
import re
from urllib.parse import urlparse
from typing import List, Dict, Any
import logging
from dotenv import load_dotenv
import os
from abc import ABC, abstractmethod
from fake_useragent import UserAgent
import html2text
from duckduckgo_search import DDGS
import random
from dataclasses import dataclass

# Load environment variables
load_dotenv()

# Google API keys
GOOGLE_CUSTOM_SEARCH_ENGINE_ID = os.getenv('GOOGLE_CUSTOM_SEARCH_ENGINE_ID')
GOOGLE_CUSTOM_SEARCH_API_KEY = os.getenv('GOOGLE_CUSTOM_SEARCH_API_KEY')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class SearchProvider(ABC):
    @abstractmethod
    def search(self, query: str, num_results: int) -> List['SearchResult']:
        """Perform a search and return a list of SearchResult objects."""

@dataclass
class SearchResult:
    title: str
    url: str
    snippet: str
    content: str = ""

class SearchAPI(SearchProvider):
    def __init__(self, name: str, api_key: str, base_url: str, 
                 params: dict, quota: int, results_path: str, 
                 rate_limit: int):
        self.name = name
        self.api_key = api_key
        self.base_url = base_url
        self.params = params.copy()
        if api_key:
            self.params['key'] = api_key
        self.quota = quota
        self.used = 0
        self.results_path = results_path
        self.rate_limit = rate_limit
        self.last_request_time = 0
        self.user_agent_rotator = UserAgent()

    def is_within_quota(self):
        return self.used < self.quota

    def respect_rate_limit(self):
        time_since_last_request = time.time() - self.last_request_time
        if time_since_last_request < self.rate_limit:
            time.sleep(self.rate_limit - time_since_last_request)

    def search(self, query: str, num_results: int) -> List[SearchResult]:
        self.respect_rate_limit()
        self.params['q'] = query
        self.params['num'] = num_results

        headers = {'User-Agent': self.user_agent_rotator.random}
        try:
            return self._extracted_from_search_8(headers)
        except requests.exceptions.RequestException as e:
            logging.error(f"Error with {self.name} API request: {type(e).__name__} - {e}")
            return []

    # TODO Rename this here and in `search`
    def _extracted_from_search_8(self, headers):
        response = requests.get(self.base_url, params=self.params, headers=headers, timeout=10)
        response.raise_for_status()
        self.used += 1
        self.last_request_time = time.time()
        data = response.json()

        results = []
        content = "No content"  # Content will be fetched separately
        for item in data.get(self.results_path, []):
            url = item.get('link') or item.get('url')
            title = item.get('title') or "No title"
            snippet = item.get('snippet') or "No snippet"
            results.append(SearchResult(title, url, snippet, content))
        return results

class DuckDuckGoSearchProvider(SearchProvider):
    def search(self, query: str, max_results: int) -> List[SearchResult]:
        try:
            sanitized_query = self._sanitize_query(query)
            with DDGS() as ddgs:
                results = list(ddgs.text(sanitized_query, region='wt-wt', safesearch='off', timelimit='y'))[:max_results]
            return [SearchResult(r['title'], r['href'], r['body']) for r in results]
        except Exception as e:
            logging.error(f"Error searching DuckDuckGo: {e}")
            return []

    def _sanitize_query(self, query: str) -> str:
        query = re.sub(r'[^\w\s]', '', query)
        query = re.sub(r'\s+', ' ', query).strip()
        return query[:5000] 

class WebContentExtractor:
    MAX_RETRIES = 3
    TIMEOUT = 10

    @staticmethod
    def extract_content(url: str) -> str:
        if not WebContentExtractor.is_valid_url(url):
            logging.error(f"Invalid URL: {url}")
            return ""
            
        user_agents = [
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
            'Mozilla/5.0 (iPhone; CPU iPhone OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Mobile/15E148 Safari/604.1',
            'Mozilla/5.0 (iPad; CPU OS 14_6 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/91.0.4472.80 Mobile/15E148 Safari/604.1',
            'Mozilla/5.0 (Android 11; Mobile; rv:68.0) Gecko/68.0 Firefox/88.0',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.59',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36 OPR/78.0.4093.147',
        ]

        headers = {
            'User-Agent': random.choice(user_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Accept-Encoding': 'gzip, deflate, br',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Sec-Fetch-User': '?1',
            'Cache-Control': 'max-age=0'
            }

        for attempt in range(WebContentExtractor.MAX_RETRIES):
            try:
                response = requests.get(url, headers=headers, timeout=WebContentExtractor.TIMEOUT)
                response.raise_for_status()
                
                content_type = response.headers.get('Content-Type', '').lower()
                if 'text/html' not in content_type:
                    logging.warning(f"Non-HTML content type for {url}: {content_type}")
                    return ""

                soup = BeautifulSoup(response.content, 'html.parser')
    
                # Remove common navigation and menu elements
                for element in soup(['nav', 'header', 'footer', 'aside']):
                    element.decompose()

                # Try to find the main content area
                content = soup.find('main') or soup.find('article') or soup.find('div', class_=['content', 'main-content', 'post-content'])

                if not content:
                    # If no main content area is found, use the body
                    content = soup.body

                if content:
                    # Remove script, style, and comment elements
                    for element in content(['script', 'style']):
                        element.decompose()
                    for comment in content.find_all(string=lambda text: isinstance(text, Comment)):
                     comment.extract()

                    # Convert to plain text
                    h = html2text.HTML2Text()
                    h.ignore_links = True
                    h.ignore_images = True
                    text = h.handle(str(content))

                    # Additional cleaning steps
                    text = re.sub(r'\n+', '\n', text)  # Remove multiple newlines
                    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
                    text = text.strip()
                else:
                    text = ""  # Set empty string if no content is found

                return text

            except requests.RequestException as e:
                logging.warning(f"Attempt {attempt + 1} failed for {url}: {e}")
                if attempt == WebContentExtractor.MAX_RETRIES - 1:
                    logging.error(f"Failed to extract content from {url} after {WebContentExtractor.MAX_RETRIES} attempts")
                    return ""
        
        return ""

    @staticmethod
    def is_valid_url(url: str) -> bool:
        try:
            result = urlparse(url)
            return all([result.scheme, result.netloc])
        except ValueError:
            return False

class SearchManager:
    def __init__(self, apis: List[SearchAPI], web_search_provider: SearchProvider, max_content_length: int = 10000, cache_size: int = 100):
        self.apis = apis
        self.web_search_provider = web_search_provider
        self.content_extractor = WebContentExtractor()
        self.max_content_length = max_content_length
        self.cache = {}
        self.cache_size = cache_size

    def search(self, query: str, num_results: int = 5) -> List[Dict]:
        if query in self.cache:
            logging.info(f"Using cached results for query: {query}")
            return self.cache[query]

        for api in self.apis:
            if api.is_within_quota():
                logging.info(f"Trying {api.name} for query: {query}")
                if search_results := api.search(query, num_results):
                    detailed_results = []
                    for result in search_results:
                        content = self.content_extractor.extract_content(result.url)
                        result.content = content[:self.max_content_length]
                        detailed_results.append({
                            'title': result.title,
                            'url': result.url,
                            'snippet': result.snippet,
                            'content': result.content
                        })

                    self.cache[query] = detailed_results
                    return detailed_results

        logging.info(f"Trying DuckDuckGo for query: {query}")
        duck_results = self.web_search_provider.search(query, num_results)
        detailed_results = []
        for result in duck_results:
            content = self.content_extractor.extract_content(result.url)
            detailed_results.append({
                'title': result.title,
                'url': result.url,
                'snippet': result.snippet,
                'content': content[:self.max_content_length]
            })

        if len(self.cache) >= self.cache_size:
            self.cache.pop(next(iter(self.cache)))  # Remove the oldest cached item

        self.cache[query] = detailed_results
        return detailed_results

class SearchProviderFactory:
    @staticmethod
    def create_provider(provider_name: str, **kwargs) -> SearchProvider:
        if provider_name.lower() == "google":
            return SearchAPI(
                "Google",
                kwargs.get('api_key', GOOGLE_CUSTOM_SEARCH_API_KEY),
                "https://www.googleapis.com/customsearch/v1",
                {"cx": kwargs.get('engine_id', GOOGLE_CUSTOM_SEARCH_ENGINE_ID)},
                kwargs.get('quota', 100),
                'items',
                kwargs.get('rate_limit', 1),
            )
        elif provider_name.lower() == "duckduckgo":
            return DuckDuckGoSearchProvider()
        else:
            raise ValueError(f"Unsupported search provider: {provider_name}")
    

# Example usage
if __name__ == "__main__":
    # Create search providers using the factory
    google_api = SearchProviderFactory.create_provider("google")
    duckduckgo_provider = SearchProviderFactory.create_provider("duckduckgo")

    # Initialize SearchManager with the providers
    search_manager = SearchManager([google_api], duckduckgo_provider)
    search_query = ""
    num_results = 10
    results = search_manager.search(search_query, num_results)
    for result in results:
        print(f"Title: {result['title']}")
        print(f"URL: {result['url']}")
        print(f"Snippet: {result['snippet']}")
        print(f"Content: {result['content'][:10000]}...")  # Print first 200 characters of content
        print("---")


